### PyWren RISECamp, 2017

Welcome to the hands-on tutorial for PyWren.

This tutorial consists of a set of exercises that will have you working directly with PyWren:
- simple matrix multiplication
- data analysis on a wikipedia dataset
- some machine learning algorithms (Eric's) 


## 0. Hello World

First, let's write a simple hello program to test out PyWren.



In [3]:
# first we need to load PyWren and creates an executor instance
import pywren
pwex = pywren.default_executor()

2017-08-22 10:09:37,777 [INFO] pywren.executor: using serializer with meta-supplied preinstalls


We can use the `call_async()` API on pywren executor to run the function in the cloud
The workflow is pretty simple and looks like this:

```python
def my_func(param):
    # do something
    return some_result
    
handler = pwex.call_async(my_func, param)
result = handler.result()
```

**Exercise**: modify the following code block to run hello world with pywren

**TODO: I think we need helper function to test against the output. This gives attendees more incentive to get things right.**

In [5]:
# first we need a basic hello world function
def hello_world(param):
    print "hello world!"

handler = pwex.call_async()
# on success, this line should print out "hello world"
print(handler.result())

2017-08-22 10:14:38,506 [INFO] pywren.executor: map 08d5a938-2aad-49ed-8dee-40aa6b11f80e 00000 apply async
2017-08-22 10:14:38,508 [INFO] pywren.executor: call_async 08d5a938-2aad-49ed-8dee-40aa6b11f80e 00000 lambda invoke 
2017-08-22 10:14:38,687 [INFO] pywren.executor: call_async 08d5a938-2aad-49ed-8dee-40aa6b11f80e 00000 lambda invoke complete
2017-08-22 10:14:38,743 [INFO] pywren.executor: map invoked 08d5a938-2aad-49ed-8dee-40aa6b11f80e 00000 pool join
2017-08-22 10:14:43,099 [INFO] pywren.future: ResponseFuture.result() 08d5a938-2aad-49ed-8dee-40aa6b11f80e 00000 call_success True


None


The above example runs a single function in the cloud.
Now PyWren also has a `map()` API that allows users to run a single function with multiple parameters:

```python
handlers = pwex.map(my_func, param_list)
pywren.wait(handlers)

results = [h.result() for h in handlers]
```

**Exercise**: modify the following code block to print "hello world"

In [7]:
# do not modify code here
def hello_world(param):
    if param == 1:
        return "hello"
    if param == 2:
        return "world!"
# do not modify code above

param_list = []
handlers = pwex.call_async(hello_world, None)

results = [h.result() for h in handlers] 
print(" ".join(results))

2017-08-22 10:42:24,308 [INFO] pywren.executor: map 4399db16-7a45-43a8-8939-2f1989d26eca 00000 apply async
2017-08-22 10:42:24,310 [INFO] pywren.executor: call_async 4399db16-7a45-43a8-8939-2f1989d26eca 00000 lambda invoke 
2017-08-22 10:42:24,311 [INFO] pywren.executor: map 4399db16-7a45-43a8-8939-2f1989d26eca 00001 apply async
2017-08-22 10:42:24,313 [INFO] pywren.executor: call_async 4399db16-7a45-43a8-8939-2f1989d26eca 00001 lambda invoke 
2017-08-22 10:42:24,459 [INFO] pywren.executor: call_async 4399db16-7a45-43a8-8939-2f1989d26eca 00001 lambda invoke complete
2017-08-22 10:42:24,462 [INFO] pywren.executor: call_async 4399db16-7a45-43a8-8939-2f1989d26eca 00000 lambda invoke complete
2017-08-22 10:42:24,535 [INFO] pywren.executor: map invoked 4399db16-7a45-43a8-8939-2f1989d26eca 00001 pool join
2017-08-22 10:42:42,251 [INFO] pywren.future: ResponseFuture.result() 4399db16-7a45-43a8-8939-2f1989d26eca 00000 call_success True
2017-08-22 10:42:42,487 [INFO] pywren.future: ResponseFutu

hello world!


## 1. Matrix Multiplication

One nice thing about PyWren is it allows users to integrate existing python libraries easily.
For the following exercise, we are going to use some popular python libraries, e.g., NumPy, to work on some matrix multiplication problems.

In [9]:
import numpy as np

def my_function(b):
    x = np.random.normal(0, b, 1024)
    A = np.random.normal(0, b, (1024, 1024))
    return np.dot(A, x)

pwex = pywren.default_executor()
res = pwex.map(my_function, np.linspace(0.1, 10, 100))


2017-08-22 10:46:58,750 [INFO] pywren.executor: using serializer with meta-supplied preinstalls
2017-08-22 10:47:00,200 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00000 apply async
2017-08-22 10:47:00,202 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00000 lambda invoke 
2017-08-22 10:47:00,203 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00001 apply async
2017-08-22 10:47:00,204 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00001 lambda invoke 
2017-08-22 10:47:00,209 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00002 apply async
2017-08-22 10:47:00,210 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00002 lambda invoke 
2017-08-22 10:47:00,214 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00003 apply async
2017-08-22 10:47:00,216 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00003 lambda invoke 
2017-08-

2017-08-22 10:47:00,434 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00013 lambda invoke complete
2017-08-22 10:47:00,440 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00015 lambda invoke complete
2017-08-22 10:47:00,448 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00030 apply async
2017-08-22 10:47:00,449 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00030 lambda invoke 
2017-08-22 10:47:00,464 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00012 lambda invoke complete
2017-08-22 10:47:00,469 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00014 lambda invoke complete
2017-08-22 10:47:00,474 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00017 lambda invoke complete
2017-08-22 10:47:00,477 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00016 lambda invoke complete
2017-08-22 10:47:00,480 [I

2017-08-22 10:47:00,713 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00054 apply async
2017-08-22 10:47:00,714 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00054 lambda invoke 
2017-08-22 10:47:00,724 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00035 lambda invoke complete
2017-08-22 10:47:00,726 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00036 lambda invoke complete
2017-08-22 10:47:00,730 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00037 lambda invoke complete
2017-08-22 10:47:00,735 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00038 lambda invoke complete
2017-08-22 10:47:00,738 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00055 apply async
2017-08-22 10:47:00,740 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00055 lambda invoke 
2017-08-22 10:47:00,751 [INFO] pywren.executor: map 

2017-08-22 10:47:00,995 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00063 lambda invoke complete
2017-08-22 10:47:01,000 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00061 lambda invoke complete
2017-08-22 10:47:01,003 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00064 lambda invoke complete
2017-08-22 10:47:01,006 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00076 apply async
2017-08-22 10:47:01,007 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00076 lambda invoke 
2017-08-22 10:47:01,015 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00065 lambda invoke complete
2017-08-22 10:47:01,019 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00077 apply async
2017-08-22 10:47:01,020 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00077 lambda invoke 
2017-08-22 10:47:01,026 [INFO] pywren.executor: map 

2017-08-22 10:47:01,256 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00100 apply async
2017-08-22 10:47:01,259 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00085 lambda invoke complete
2017-08-22 10:47:01,261 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00100 lambda invoke 
2017-08-22 10:47:01,264 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00101 apply async
2017-08-22 10:47:01,265 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00101 lambda invoke 
2017-08-22 10:47:01,269 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00086 lambda invoke complete
2017-08-22 10:47:01,278 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00087 lambda invoke complete
2017-08-22 10:47:01,278 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00102 apply async
2017-08-22 10:47:01,279 [INFO] pywren.executor: call_async 4c03cca5-ff

...

2017-08-22 10:47:10,779 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00904 apply async
2017-08-22 10:47:10,781 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00904 lambda invoke 
2017-08-22 10:47:10,788 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00889 lambda invoke complete
2017-08-22 10:47:10,792 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00905 apply async
2017-08-22 10:47:10,793 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00905 lambda invoke 
2017-08-22 10:47:10,799 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00906 apply async
2017-08-22 10:47:10,800 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00906 lambda invoke 
2017-08-22 10:47:10,804 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00907 apply async
2017-08-22 10:47:10,805 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 

2017-08-22 10:47:11,068 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00927 lambda invoke 
2017-08-22 10:47:11,077 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00913 lambda invoke complete
2017-08-22 10:47:11,079 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00928 apply async
2017-08-22 10:47:11,081 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00928 lambda invoke 
2017-08-22 10:47:11,090 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00915 lambda invoke complete
2017-08-22 10:47:11,095 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00914 lambda invoke complete
2017-08-22 10:47:11,100 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00916 lambda invoke complete
2017-08-22 10:47:11,104 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00929 apply async
2017-08-22 10:47:11,105 [INFO] pywren.executor: call

2017-08-22 10:47:11,392 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00950 apply async
2017-08-22 10:47:11,393 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00950 lambda invoke 
2017-08-22 10:47:11,400 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00951 apply async
2017-08-22 10:47:11,405 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00938 lambda invoke complete
2017-08-22 10:47:11,407 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00951 lambda invoke 
2017-08-22 10:47:11,413 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00952 apply async
2017-08-22 10:47:11,414 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00952 lambda invoke 
2017-08-22 10:47:11,421 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00941 lambda invoke complete
2017-08-22 10:47:11,430 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-

2017-08-22 10:47:11,686 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00978 lambda invoke 
2017-08-22 10:47:11,692 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00979 apply async
2017-08-22 10:47:11,695 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00953 lambda invoke complete
2017-08-22 10:47:11,696 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00979 lambda invoke 
2017-08-22 10:47:11,705 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00952 lambda invoke complete
2017-08-22 10:47:11,710 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00951 lambda invoke complete
2017-08-22 10:47:11,714 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00963 lambda invoke complete
2017-08-22 10:47:11,715 [INFO] pywren.executor: map 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00980 apply async
2017-08-22 10:47:11,721 [INFO] pywren.executor: call

2017-08-22 10:47:11,984 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00981 lambda invoke complete
2017-08-22 10:47:11,997 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00982 lambda invoke complete
2017-08-22 10:47:12,005 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00984 lambda invoke complete
2017-08-22 10:47:12,009 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00983 lambda invoke complete
2017-08-22 10:47:12,022 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00985 lambda invoke complete
2017-08-22 10:47:12,060 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00986 lambda invoke complete
2017-08-22 10:47:12,063 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00988 lambda invoke complete
2017-08-22 10:47:12,067 [INFO] pywren.executor: call_async 4c03cca5-ff3c-4063-9f31-8b57e06b132b 00990 lambda invoke complete


## 2. Data Analytics with Wikipedia Dataset

## 3. Some Machine Learning